In [1]:
#Import Dependencies

from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np

from time import sleep
from random import randint

import json

In [2]:
# Sets a path to Google Chrome
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/joescuteri/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


In [3]:
URL = ['https://twitchtracker.com/games', 'https://twitchtracker.com/games?page=2', 'https://twitchtracker.com/games?page=3', 'https://twitchtracker.com/games?page=4', 'https://twitchtracker.com/games?page=5']
tags = []
games = []
cleanedviews = []
views = []

for url in range(0,5):      
    browser.visit(URL[url])
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    href_tags = soup.find_all(href=True)
    raw_games = href_tags[40:60]
    
    for x in raw_games:
        tags.append(str(x))
    
    for element in tags:
        if element not in games:
            front = element.split(">")[1]
            back = front.split("<")[0]
            games.append(back)
        
    viewhtml = soup.find_all('div', class_="ri-value")
    
    for x in viewhtml:
        cleanedviews.append(str(x))
    

    for element in cleanedviews:
        if element not in views:
            front = element.split(">")[2]
            back = front.split("<")[0]
            views.append(back)
            
    print(f'Page Done!')
    
    sleep(randint(5,10))

Page Done!
Page Done!
Page Done!
Page Done!
Page Done!


In [4]:
# Upload to DataFrame 
df = pd.DataFrame(games, columns = ['GAME'])
df['VIEWS'] = views
df

,GAME,VIEWS
0,Just Chatting,391K
1,League of Legends,281K
2,Grand Theft Auto V,213K
3,VALORANT,119K
4,Fortnite,106K
...,...,...
295,Farming Simulator 22,3.0K
296,MapleStory,3.0K
297,Age of Empires IV,2.9K
298,Tibia,2.9K


In [5]:
# Change VIEWS column to integers

df.VIEWS = (df.VIEWS.replace(r'[KM]+$', '', regex=True).astype(float) * df.VIEWS.str.extract(r'[\d\.]+([KM]+)', expand=False)
             .fillna(1)
             .replace(['K','M'], [10**3, 10**6]).astype(int))

In [6]:
avg_views = df['VIEWS'].mean()
median_views = df['VIEWS'].median()

In [7]:
df

,GAME,VIEWS
0,Just Chatting,391000.0
1,League of Legends,281000.0
2,Grand Theft Auto V,213000.0
3,VALORANT,119000.0
4,Fortnite,106000.0
...,...,...
295,Farming Simulator 22,3000.0
296,MapleStory,3000.0
297,Age of Empires IV,2900.0
298,Tibia,2900.0


In [8]:
df.drop(df.tail(1).index,inplace=True)

In [9]:
df

,GAME,VIEWS
0,Just Chatting,391000.0
1,League of Legends,281000.0
2,Grand Theft Auto V,213000.0
3,VALORANT,119000.0
4,Fortnite,106000.0
...,...,...
294,Stardew Valley,3000.0
295,Farming Simulator 22,3000.0
296,MapleStory,3000.0
297,Age of Empires IV,2900.0


In [51]:
games_list = df['GAME'].tolist()
views_list = df['VIEWS'].tolist()

In [52]:
game_json_list = json.dumps(games_list)
views_json_list = json.dumps(views_list)

In [69]:
games_list

['Just Chatting', 'League of Legends', 'Grand Theft Auto V', 'VALORANT', 'Fortnite', 'Apex Legends', 'Rust', 'Minecraft', 'Escape from Tarkov', 'Dota 2', 'Call of Duty: Warzone', 'Counter-Strike: Global Offensive', 'FIFA 22', 'God of War', 'Slots', 'Teamfight Tactics', 'Music', 'Monster Hunter Rise', 'Hearthstone', 'Dead by Daylight', 'Just Chatting', 'League of Legends', 'Grand Theft Auto V', 'VALORANT', 'Fortnite', 'Apex Legends', 'Rust', 'Minecraft', 'Escape from Tarkov', 'Dota 2', 'Call of Duty: Warzone', 'Counter-Strike: Global Offensive', 'FIFA 22', 'God of War', 'Slots', 'Teamfight Tactics', 'Music', 'Monster Hunter Rise', 'Hearthstone', 'Dead by Daylight', 'Special Events', 'PUBG: BATTLEGROUNDS', 'Yu-Gi-Oh! Master Duel', 'Rocket League', 'Sports', 'Genshin Impact', 'LOST ARK', 'ASMR', "Tom Clancy's Rainbow Six Extraction", 'Chess', 'Art', 'Overwatch', 'Old School RuneScape', 'Clash Royale', 'Talk Shows &amp; Podcasts', 'Final Fantasy XIV Online', 'Virtual Casino', 'Red Dead Red

In [66]:
game_array = df[["GAME"]].to_numpy()
views_array= df[["VIEWS"]].to_numpy()

In [33]:
gamesjson = df.to_json()
json_object = json.loads(gamesjson)
json_formatted_str = json.dumps(json_object, indent=2)
# print(json_formatted_str)

In [32]:
aDict = json.loads(json_formatted_str)
# print(aDict)

In [30]:
# print(aDict['GAME'])

In [31]:
# print(aDict['VIEWS'])

In [10]:
# Convert DataFrame into Dictionary
twitch = pd.Series(df.VIEWS.values,index=df.GAME).to_dict()

In [11]:
# twitch

# Migrating it to MongoDB

In [13]:
# connecting to mongodb
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# declare the database
db = client.twitchviews_db
collection = db.items

In [14]:
# Push into MongoDB
collection.insert_one(twitch)